In [31]:

from tensorflow import keras
import tensorflow as tf
import numpy as np
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig,DistilBertTokenizer,TFDistilBertForSequenceClassification
'''
Predictor class 
Allows to create an object to run new data on 
path to pretrained tokinizers and model are required
'''
class Predictor:

  def __init__(self,path_tokenizer,path_model):
    self.tokenizer = DistilBertTokenizer.from_pretrained(path_tokenizer,local_files_only=True)
    self.model = TFDistilBertForSequenceClassification.from_pretrained('./model_BERT')

  '''
  runs predictions for a given sentance
  '''
  def pred_vectorized(self,text,max_length = 150):
      preds = self.tokenizer.encode(text,truncation=True,padding=True,max_length=max_length,return_tensors="tf")
      pred =  self.model.predict(preds)[0]
      softed = tf.nn.softmax(pred).numpy()[0]
      out = np.where( np.round(softed) ==1)[0][0]
      return out

  '''
  Predicts and runs a list comprehension to predict values

  +++++++++++++++++++++++++++++
  Find a way to do this with numpy vectorization
  +++++++++++++++++++++++++++++
  
  '''
  def predict_prod(self,path):

    prod_data = load_dataset('csv', data_files=path)
    prod_txt = prod_data['train']['0']
    preds_prod = [self.pred_vectorized(x) for x in prod_txt] 
    df = pd.read_csv('/content/prod_data.csv',index_col = [0])
    df['out'] = pd.Series(preds_prod).values
    self.prod_predict = df 
    return df 

  def save_df(self,path):
    self.prod_predict.to_csv(path)




In [32]:
model_BERT = Predictor(path_tokenizer='./tokenizer',path_model='./model')

Some layers from the model checkpoint at ./model_BERT were not used when initializing TFDistilBertForSequenceClassification: ['dropout_116']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./model_BERT and are newly initialized: ['dropout_57']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': TensorSpec(shape=(None, 5), dtype=tf.int32, name='input_ids/input_ids')}

In [33]:
pred = model_BERT.pred_vectorized('you will need to know sql')


C:\Users\Emiliano\Anaconda3\envs\DataSci\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
